<div class="alert alert-block alert-warning"><strong>If you have this notebook as a local copy on your platform, it may become outdated. <a href="https://dataplatform.cloud.ibm.com/exchange/public/entry/view/029d77a73d72a4134c81383d6f01f1ed?context=cpdaas&audience=wdp">Download the latest version of this notebook</a> or download the <a href="https://dataplatform.cloud.ibm.com/exchange/public/entry/view/cab78523832431e767c41527a42a6727?context=cpdaas?context=cpdaas&audience=wdp">latest version of the project</a>.</strong></div>

# Part 1 - WML Federated Learning with XGBoost and Adult Income dataset - Aggregator 

With IBM Federated Learning, you can combine data from multiple sources to train a model from the collective data without having to actually share them. This allows enterprises to train data with other companies without delegating resources for security. Another advantage is the remote data does not have to be centralized in one location, eliminates the needs to move potentially large datasets. This notebook demonstrates how to start Federated Learning with the Python client. For more details setting up Federated Learning, terminology, and running Federated Learning from the UI, see [Federated Learning documentation](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fed-lea.html?audience=wdp).

### Learning Goals

When you complete the Part 1 - WML Federated Learning with XGBoost and Adult Income dataset - Aggregator notebook, you should know how to:

- Create a Remote Training System
- Start a training job

Once you complete this notebook, please open [Part 2 - WML Federated Learning with XGBoost and Adult Income dataset - Party](https://dataplatform.cloud.ibm.com/). 

<div class="alert alert-block alert-info">This notebook is intended to be run by the administrator of the Federated Learning experiment.</div>

## Table of Contents

- [1. Prequisites](#prequisites)
    - [1.2 Define variables](#var)
    - [1.3 Import libraries](#libraries)
    - [1.3 Obtain Cloud authentication token](#cloud-auth)
- [2. Load the model](#load)
    - [2.1 Create Untrained Model Asset](#model)
    - [2.2 Upload Initial Model](#init)
- [3. Create a Remote Training System](#rts)
    - [3.1 Create Remote Training System Asset](#create-rts)
- [4. Create FL Training Job](#fl-job)
    - [4.1 Get Training Job Status](#status)

<a id = "prequisites"></a>
## 1. Prequisites

Before you proceed, you need to have:

- An IAM API Key. To create a new one, go to [IBM Cloud homepage](https://cloud.ibm.com). In your account, go to **Manage < IAM < API Keys**. Click **Create an IBM Cloud API Key**.

<a id = "var"></a>
### 1.1 Define variables

In [ ]:
API_VERSION = "2021-10-01"

WML_SERVICES_HOST = "us-south.ml.cloud.ibm.com" # or "eu-de.ml.cloud.ibm.com"
WML_SERVICES_URL = "https://" + WML_SERVICES_HOST
IAM_TOKEN_URL = "https://iam.cloud.ibm.com/oidc/token"
 
IAM_APIKEY = "XXX"  

# Get this from Manage < IAM < Users, and check the URL. Your user ID should be in the format IBMid-<xxx>.
CLOUD_USERID = "IBMid-XXX" 

PROJECT_ID = "XXX" # Get this by going into your WS project and checking the URL.

<a id = "libraries"></a>
### 1.2 Import libraries

In [ ]:
import urllib3
import requests
import json
from string import Template

urllib3.disable_warnings()

<a id = "auth"></a>
## 2. Obtain Cloud authentication token

In [ ]:
payload = "grant_type=urn:ibm:params:oauth:grant-type:apikey&apikey=" + IAM_APIKEY
token_resp = requests.post(IAM_TOKEN_URL ,
                          headers={"Content-Type": "application/x-www-form-urlencoded"}, 
                          data = payload,
                          verify=True)

print(token_resp)

token = "Bearer " + json.loads(token_resp.content.decode("utf-8"))["access_token"]
print("WS token: %s " % token)

<a id = "create-rts"></a>
## 3. Create Remote Training System Asset

Now you will learn to create a Remote Training System (RTS). An RTS handles receiving your multiple parties' call to the aggregator to run the training. 
- `allowed_identities` are users permitted to connect to the Federated Learning experiment.  In this tutorial, only your user ID is permitted to connect but you can update the template and add additional users as required.
- An Admin in `remote_admin`. The template for the admin is the same as the user. In this tutorial, a template Admin is created. It is also the same as the user ID, however generally in application, the admin does not have to be one of the users.

In [ ]:
wml_remote_training_system_asset_one_def = Template("""
{
  "name": "Remote Party 1",
  "project_id": "$projectId",
  "description": "Sample Remote Training System",
  "tags": [ "Federated Learning" ],
  "organization": {
    "name": "IBM",
    "region": "US"
  },
  "allowed_identities": [
    {
      "id": "$userID",
      "type": "user"
    }
  ],
  "remote_admin": {
    "id": "$userID",
    "type": "user"
  }
}
""").substitute(userID = CLOUD_USERID,
                projectId = PROJECT_ID)


wml_remote_training_system_one_resp = requests.post(WML_SERVICES_URL + "/ml/v4/remote_training_systems", 
                                                    headers={"Content-Type": "application/json",
                                                             "Authorization": token}, 
                                                    params={"version": API_VERSION,
                                                            "project_id": PROJECT_ID}, 
                                                    data=wml_remote_training_system_asset_one_def, 
                                                    verify=False)

print(wml_remote_training_system_one_resp)
status_json = json.loads(wml_remote_training_system_one_resp.content.decode("utf-8"))
print("Create remote training system response : "+ json.dumps(status_json, indent=4))

wml_remote_training_system_one_asset_uid = json.loads(wml_remote_training_system_one_resp.content.decode("utf-8"))["metadata"]["id"]
print("Remote Training System id: %s" % wml_remote_training_system_one_asset_uid)

<a id = "fl-job"></a>
## 4. Create FL Training Job

In this section, you will launch the Federated Learning experiment.

In [ ]:
training_payload = Template(""" 
{
  "name": "FL Aggregator",
  "tags": [
    {
      "value": "tags_jobs_fl",
      "description": "Sample FL Aggregator"
    }
  ],
  "federated_learning": {
    "fusion_type": "xgb_classifier",
    "learning_rate": 0.1,
    "loss": "binary_crossentropy",
    "max_bins": 255,
    "rounds": 3,
    "num_classes": 2,
    "metrics": "loss",
    "remote_training" : {
      "quorum": 1.0,
      "remote_training_systems": [ { "id" : "$rts_one", "required" : true  } ]
    },
    "hardware_spec": {
      "name": "XS"
    }
  },
  "training_data_references": [],
  "results_reference": {
    "type": "container",
    "name": "outputData",
    "connection": {},
    "location": {
      "path": "."
    }
  },
  "project_id": "$projectId"  
}
""").substitute(projectId = PROJECT_ID,
                rts_one = wml_remote_training_system_one_asset_uid)

create_training_resp = requests.post(WML_SERVICES_URL + "/ml/v4/trainings", params={"version": API_VERSION},
                                     headers={"Content-Type": "application/json",
                                              "Authorization": token},
                                     data=training_payload,
                                     verify=False)

print(create_training_resp)
status_json = json.loads(create_training_resp.content.decode("utf-8"))
print("Create training response : "+ json.dumps(status_json, indent=4))

training_id = json.loads(create_training_resp.content.decode("utf-8"))["metadata"]["id"]
print("Training id: %s" % training_id)

<a id = "status"></a>
### 4.1 Get Training Job Status


In [ ]:
get_training_resp = requests.get(WML_SERVICES_URL + "/ml/v4/trainings/" + training_id,
                                 headers={"Content-Type": "application/json",
                                          "Authorization": token},
                                  params={"version": API_VERSION,
                                          "project_id": PROJECT_ID},
                                  verify=False)

print(get_training_resp)
status_json = json.loads(get_training_resp.content.decode("utf-8"))
print("Get training response : "+ json.dumps(status_json, indent=4))

### Wait For Aggregator Status To Become Accepting Parties

<div class="alert alert-block alert-warning">Before you proceed further, run the previous cell. The output should be `accepting_parties`. If output is `pending`, wait for a few minutes and re-run the <i>previous</i> code cell, with <code>requests.get</code>, and then re-run this code cell again to see the output.
</div>

## 5. Get Variables And Paste Into Party Notebook

Run the following cell and copy the output. 

In [ ]:
print("WML_SERVICES_HOST = '%s'" % WML_SERVICES_HOST)
print("IAM_APIKEY = '%s'" % IAM_APIKEY)
print("RTS_ID = '%s'" % wml_remote_training_system_one_asset_uid)
print("TRAINING_ID = '%s'" % (training_id))

As the Admin, you have now launched a Federated Learning experiment. Copy the output from the previous cell. Open Part 2 - WML Federated Learning with XGBoost and Adult Income dataset - Party and paste the output into the first code cell.

# <hr>
Copyright © 2020 IBM. This notebook and its source code are released under the terms of the MIT License.
 <br><br>
<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>